In [1]:
from matplotlib import pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pysr import PySRRegressor

In [2]:
df = pd.read_fwf('mass.mas03', usecols=(2, 3, 11),
                 widths=(1, 3, 5, 5, 5, 1, 3, 4, 1, 13, 11, 11,
                         9, 1, 2, 11, 9, 1, 3, 1, 12, 11, 1),
                 skiprows=41, header=None,
                 index_col=False)
df.columns = ('n', 'Z',  'avEbind')

# Extrapolated values are indicated by '#' in place of the decimal place, so
# the avEbind column won't be numeric. Coerce to float and drop these entries.
df['avEbind'] = pd.to_numeric(df['avEbind'], errors='coerce')
df = df.dropna()
df['A'] = df.n + df.Z
X = df[['A', 'n', 'Z']]
y = df[['avEbind']]

X

,A,n,Z
0,2,1,1
1,3,2,1
2,3,1,2
4,4,3,1
5,4,2,2
...,...,...,...
3010,258,157,101
3029,260,154,106
3034,261,157,104
3058,264,156,108


In [ ]:
model = PySRRegressor(
    update=False,
    multithreading=True,
    niterations=100,
    populations=100,
    binary_operators=["plus", "mult", "pow"],
    unary_operators=[
        "cos",
        "exp",
        "sin",
        "log"
        "odd(x) = isodd(x) ? 1.0f0 : -1.0f0"
    ],
    extra_sympy_mappings={'special': lambda x, y: x**2 + y},
    model_selection="best", # "accuracy"
    loss="f(x, y) = (x - y)^2",  # Custom loss function,
    constraints={'pow': (-1, 3), 'mult': (3, 3), 'cos': 5}
)

model.fit(X, y)
print(model)

    # procs=20,
    # "inv(x) = 1/x",  # Custom operator (julia syntax)

/root/.pyenv/versions/3.9.10/lib/python3.9/site-packages/pysr/sr.py:1113: UserWarning: Resetting variable_names from X.columns
  warnings.warn("Resetting variable_names from X.columns")
  Activating project at `~/.julia/environments/pysr-0.8.7`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/pysr-0.8.7/Project.toml`
  No Changes to `~/.julia/environments/pysr-0.8.7/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/pysr-0.8.7/Project.toml`
  No Changes to `~/.julia/environments/pysr-0.8.7/Manifest.toml`


In [ ]:
eqs=model.equations
be=eqs.score.idxmax()
l=eqs.loss[be]
s=eqs.score[be]
e=eqs.sympy_format[be]
ef=eqs.lambda_format[be]
print ("best result:\nloss:",l,'  score:', s, '  eq:', e )

In [ ]:
df["pred"]=ef(X)
df["res"]= df.avEbind - df.pred

In [ ]:
pdf=df.drop(['avEbind','A'], axis = 1)
tp=pdf.set_index(['Z', 'n']).res.unstack(0)

fig, ax = plt.subplots(figsize=(7, 5), dpi=150)
z_min=pdf.res.min()
z_max=pdf.res.max()
print(z_min, z_max)

# c = ax.pcolormesh(tp, cmap= cm.coolwarm, norm=colors.LogNorm(vmin=z_min, vmax=z_max)) 
c=ax.pcolormesh(tp, norm=colors.SymLogNorm(linthresh=0.1, linscale=0.1, vmin=z_min, vmax=z_max, base=10),
                       cmap=cm.coolwarm, shading='auto')

ax.set_title('optimized')
# set the limits of the plot to the limits of the data
ax.axis([pdf.Z.min(), pdf.Z.max(), pdf.n.min(), pdf.n.max()])
fig.colorbar(c, ax=ax)

ax.set_xlabel('Z')
ax.set_ylabel('N')

plt.show()
plt.savefig("prediction.png", dpi=150)